# The Diet Problem

In [1]:
#!pip install numpy
#!pip install wheel
#!pip install pandas
import gurobipy as grb
import scipy.sparse as sp
import pandas as pd
import numpy as np
import os

Have a look at the diet problem data:

In [2]:
filename = 'data_mec_optim/lp_stigler-diet/StiglerData1939.txt'
thepath = os.path.join(os.getcwd(),'..')
thedata = pd.read_csv(os.path.join(thepath ,filename), sep='\t')
thedata = thedata.dropna(how = 'all')
thedata

,Commodity,Unit,Price Aug.15 1939(cents),Edible Weight per $1.00 (grams),Calories (1000),Protein(grams),Calcium(grams),Iron(mg.),Vitamin A(1000 I.U),Thiamine(mg.),Riboflavin(mg.),Niacin(mg.),Asorbic Acid (mg.)
0,1. Wheat Flour (Enriched),10 lb.,36.0,12600.0,44.7,1411.0,2.0,365.0,NaN,55.4,33.3,441.0,NaN
1,2. Macaroni,1 lb.,14.1,3217.0,11.6,418.0,0.7,54.0,NaN,3.2,1.9,68.0,NaN
2,3. Wheat Cereal (Enriched),28 oz.,24.2,3280.0,11.8,377.0,14.4,175.0,NaN,14.4,8.8,114.0,NaN
3,4. Corn Flakes,8 oz.,7.1,3194.0,11.4,252.0,0.1,56.0,NaN,13.5,2.3,68.0,NaN
4,5. Corn Meal,1 lb.,4.6,9861.0,36.0,897.0,1.7,99.0,30.9,17.4,7.9,106.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,74. Sugar,10 lb.,51.2,8773.0,34.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,75. Corn Sirup,24 oz.,13.7,4966.0,14.7,NaN,0.5,74.0,NaN,NaN,NaN,5.0,NaN
75,76. Molasses,18 oz.,13.6,3752.0,9.0,NaN,10.3,244.0,NaN,1.9,7.5,146.0,NaN
76,77. Strawberry Preserves,1 lb.,20.5,2213.0,6.4,11.0,0.4,7.0,0.2,0.2,0.4,3.0,NaN


Isolate model inputs:

In [3]:
commodity = (thedata['Commodity'].values)[:-1]
intake = thedata.iloc[:-1, 4:].fillna(0).transpose().values
allowance = thedata.iloc[-1, 4:].fillna(0).transpose()

Set up the model:Set up the model: prior to version 9.0, Gurobi in Python was different from R, in the sense that we could not input a matrix of constraints: note that Gurobi in Python is different from R, in the sense that we cannot a matrix of constraints

In [4]:
m = grb.Model('optimalDiet')
meal = m.addVars(commodity, name='meal')
m.setObjective(meal.sum(), grb.GRB.MINIMIZE)
m.addConstrs((grb.quicksum(meal[k] * intake[i, j] for j, k in enumerate(commodity)) >= allowance[i]
              for i in range(intake.shape[0])), name='c')

Using license file C:\Users\jmcgn\gurobi.lic
Academic license - for non-commercial use only


{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>}

In [5]:
m.optimize()
if m.status == grb.GRB.Status.OPTIMAL:
    total = 0
    solution = m.getAttr('x', meal)
    pi = m.getAttr('pi')

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 9 rows, 77 columns and 570 nonzeros
Model fingerprint: 0x7ae6b743
Coefficient statistics:
  Matrix range     [1e-01, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-01, 8e+01]
Presolve removed 0 rows and 47 columns
Presolve time: 0.01s
Presolved: 9 rows, 30 columns, 240 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.384688e+01   0.000000e+00      0s
       5    1.0866228e-01   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.01 seconds
Optimal objective  1.086622782e-01


In [6]:
print('***Optimal solution***')
for food in commodity:
    if solution[food] > 0:
        print(food, solution[food] * 365)
        total += solution[food] * 365
print('Total cost (optimal) =', total)

***Optimal solution***
1. Wheat Flour (Enriched) 10.774457511918214
30. Liver (Beef) 0.6907834111074215
46. Cabbage 4.0932688648428766
52. Spinach  1.8277960703546998
69. Navy Beans Dried 22.27542568724304
Total cost (optimal) = 39.66173154546625


Since version 9.0, Gurobi now allows passing a matrix of constraints from Python:

In [7]:
m2 = grb.Model('optimalDietMatrix')
x = m2.addMVar(shape=commodity.shape, name="x")
m2.setObjective(x.sum(), grb.GRB.MINIMIZE)
m2.addConstr(sp.csr_matrix(intake) @ x >= np.array(allowance), name="c")
m2.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 9 rows, 77 columns and 570 nonzeros
Model fingerprint: 0x7ae6b743
Coefficient statistics:
  Matrix range     [1e-01, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-01, 8e+01]
Presolve removed 0 rows and 47 columns
Presolve time: 0.01s
Presolved: 9 rows, 30 columns, 240 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   1.384688e+01   0.000000e+00      0s
       5    1.0866228e-01   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.02 seconds
Optimal objective  1.086622782e-01


As promised, we achieve the minimum cost bundle at $\$39.67$ per year in $1939$ dollars. If we compare this to Stigler's solutions which was 

|Food| Annual Quantities| Annual Cost|
| ---------- | ------------------ | ------------ |
| Wheat Flour | 	370 lb.|   \$13.33 |
| Evaporated Milk | 	57 cans |	  \$3.84 |
|Cabbage| 	111 lb. 	  |\$4.11|
|Spinach| 	23 lb. 	  |\$1.85|
|Dried Navy Beans| 	285 lb. |	\$16.80|
|Total Annual Cost| 	&nbsp; 	| \$39.93 |